In [1]:
import ee 
import pandas 


In [2]:
ee.Initialize()

### For ROI in India and China

In [4]:
polygon_coords = [[
        [102.1445, 23.3937],
        [109.4462, 23.3937],
        [109.4462, 8.3439],
        [102.1445, 8.3439],
        [102.1445, 23.3937]
    ]]

"""{"type":"Polygon","coordinates":[[
[37.95826023542673,-1.230450106759613],
[37.95762785043055,-1.447358160448246],
[37.741352181738094,-1.447990545444423],
[37.74261695173045,-1.229817721763436],
[37.95826023542673,-1.230450106759613]]]}"""

region = ee.Geometry.Polygon(polygon_coords)

start_date = '2017-01-01'
end_date = '2022-12-31'

# Create a point geometry 

# Create a Sentinel-1 image collection
collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(region) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('platform_number', 'A')) \
    .select(['VV', 'VH']) 


def extract_metadata(image):
    return ee.Feature(None, {
        'date': image.date().format('YYYY-MM-dd'),
        'orbit_pass': image.get('orbitProperties_pass')
    })

features = collection.map(extract_metadata).distinct(['date', 'orbit_pass'])
date = features.aggregate_array('date').getInfo()
orbit_pass = features.aggregate_array('orbit_pass').getInfo()

# Create a DataFrame from the metadata
df = pandas.DataFrame({
    'date': date,
    'orbit_pass': orbit_pass
})

df['time'] = df['date'].astype(str)
df = df.sort_values(by='date', ascending=True)

df.to_csv('vietnam_s1_metadata.csv', index=False)

### For 1km data

#### Get s1 dates for grid cells

In [ ]:
import ee
import geopandas as gpd
import json
import os
import time
# Authenticate and initialize Earth Engine
ee.Initialize()

# === INPUT ===
grid_file = "1km/filtered_grid_40km_with_points.gpkg"  # Must contain a 'grid_id' column
output_dir = "1km/s1_dates_per_grid"
os.makedirs(output_dir, exist_ok=True)

# === Load grid geometries ===
grid_gdf = gpd.read_file(grid_file).to_crs("EPSG:4326")

# === Helper function ===
def get_s1_dates(geom, start_date, end_date, orbit_pass):
    ee_geom = ee.Geometry(geom.__geo_interface__)
    s1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterDate(start_date, end_date) \
        .filterBounds(ee_geom) \
        .filter(ee.Filter.eq("instrumentMode", "IW")) \
        .filter(ee.Filter.eq("orbitProperties_pass", orbit_pass)) \
        .select(["VV", "VH"])
    
    dates = s1.aggregate_array("system:time_start").getInfo()
    unique_dates = sorted(set([
        ee.Date(d).format("YYYY-MM-dd").getInfo() for d in dates
    ]))
    return unique_dates

# === Process each grid cell ===
for _, row in grid_gdf.iterrows():
    grid_id = row["id"]
    geom = row["geometry"]
    if os.path.exists(os.path.join(output_dir, f"s1_dates_{grid_id}.json")):
        continue
    try:
        ascending_dates = get_s1_dates(geom, "2021-01-01", "2022-12-31", "ASCENDING")
        descending_dates = get_s1_dates(geom, "2021-01-01", "2022-12-31", "DESCENDING")

        if not ascending_dates and not descending_dates:
            print(f"No Sentinel-1 data for {grid_id}")
            continue

        out_data = {
            "grid_id": grid_id,
            "ascending": ascending_dates,
            "descending": descending_dates
        }

        out_path = os.path.join(output_dir, f"s1_dates_{grid_id}.json")
        with open(out_path, "w") as f:
            json.dump(out_data, f)

        print(f"Saved dates for {grid_id}: {len(ascending_dates)} ASC, {len(descending_dates)} DESC")

        time.sleep(5)

    except Exception as e:
        print(f"Failed for {grid_id}: {e}")
